In [5]:
import numpy as np
import scipy
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import product
import os
from utils import get_sample_freq, get_test_freq, get_stimulus, remove_outliers

mode = 'Frequency'
starting_point = 1.0
window_size = 2.5

sample_path = f'./sample_{mode}/data_sp{starting_point}ws{window_size}/sample_spike_neuron'
test_path = f'./test_{mode}/data_sp{starting_point}ws{window_size}/test_spike_case'

#starting_point_types = [0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.]
#window_size_types = [1,1.5,2,2.5,3,3.5,4]

stimulus_raw = scipy.io.loadmat('dat_stimulus.mat')
spk_sample_raw = scipy.io.loadmat('dat_spk_sample.mat')
spk_test_raw = scipy.io.loadmat('dat_spk_test.mat')

stimulus = stimulus_raw['stimulus']
spk_sample = spk_sample_raw['spk_sample']
spk_test = spk_test_raw['spk_test']

degrees = np.unique(stimulus[:,1])
print(degrees)

[  0.  20.  40.  60.  80. 100. 120. 140. 160.]


In [6]:
def get_data(i, type = 'Neuron'):
    if type.lower() == 'neuron':
        return pd.read_csv(sample_path + f'{i}.csv')
    elif type.lower() == 'case':
        return pd.read_csv(test_path + f'{i}.csv')

In [12]:
##make_binomial
def get_likelihood(df, percentile_threshold = 50):
    result = {}
    thr = np.percentile(df[mode].values, q = percentile_threshold)
    for degree in degrees:
        l = (df[df['Degree'] == int(degree)][mode].values > thr)*1
        result[int(degree)] = l
    return result

Neurons = {}
for n in np.arange(1,11):
    df = get_data(n, type = 'Neuron')
    Neurons[n] = get_likelihood(df)

Cases = {}
for n in np.arange(1,11):
    df = get_data(n, type = 'Case')
    Cases[n] = get_likelihood(df)

Bayes rule: posterior = likelihood * prior / evidence
evidence 는 구하기 힘듦으로 보통 posterior 가 likelihood * prior에 비례한다고 많이 함.

Beta Distribution & Binomial-Beta conjugacy 

Conjugacy prior: 

In [21]:
from math import sqrt

a0 = 2.
b0 = 2.

def get_posterior(data_dict, a0 = 2., b0 = 2.):
    Mean = []
    Std = []

    for i in data_dict:
        l = data_dict[i]
        l_mean = []
        l_std = []
        for degree in degrees:
            n = len(l[degree])
            s = sum(l[degree])
            f = n - s

            a_post = a0 + s
            b_post = b0 + f

            mu_post = a_post / (a_post + b_post)
            sig_post = sqrt((a_post*b_post)/(((a_post + b_post)**2)*(a_post + b_post + 1)))

            l_mean.append(mu_post)
            l_std.append(sig_post)
        Mean.append(l_mean)
        Std.append(l_std)

    return np.array(Mean), np.array(Std)

In [22]:
Case_mean, Case_std = get_posterior(Cases)
Neuron_mean, Neuron_std = get_posterior(Neurons)

In [38]:
Case_mean

array([[0.2       , 0.18181818, 0.5       , 0.7       , 0.81818182,
        0.81818182, 0.63636364, 0.41666667, 0.2       ],
       [0.8       , 0.54545455, 0.6       , 0.3       , 0.27272727,
        0.36363636, 0.36363636, 0.33333333, 0.7       ],
       [0.3       , 0.45454545, 0.4       , 0.4       , 0.63636364,
        0.63636364, 0.63636364, 0.58333333, 0.4       ],
       [0.2       , 0.36363636, 0.5       , 0.6       , 0.72727273,
        0.72727273, 0.54545455, 0.25      , 0.2       ],
       [0.8       , 0.72727273, 0.7       , 0.5       , 0.27272727,
        0.36363636, 0.27272727, 0.41666667, 0.4       ],
       [0.3       , 0.36363636, 0.6       , 0.7       , 0.81818182,
        0.72727273, 0.63636364, 0.16666667, 0.2       ],
       [0.2       , 0.27272727, 0.3       , 0.5       , 0.81818182,
        0.81818182, 0.72727273, 0.41666667, 0.3       ],
       [0.5       , 0.27272727, 0.4       , 0.2       , 0.27272727,
        0.36363636, 0.72727273, 0.83333333, 0.8       ],


In [37]:
from itertools import permutations
from tqdm import tqdm
min_ls_error = np.inf
P_ls_2 = None

for perm in tqdm(permutations(range(10))):
    P = np.eye(10)[list(perm)]
    ls_error = np.linalg.norm(Case_mean - P @ Neuron_mean)
    if ls_error < min_ls_error:
        min_ls_error = ls_error
        P_ls_2 = P
    
print('Least Square Perm Matrix')
print(P_ls_2)


3628800it [01:41, 35791.47it/s]

Least Square Perm Matrix
[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


In [34]:
from numpy.linalg import pinv
P_pseudo = Case_mean @ pinv(Neuron_mean)

In [36]:
np.argmax(P_pseudo, 1)

array([5, 2, 6, 5, 8, 7, 2, 3, 5, 9])